In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift, estimate_bandwidth
import pymeanshift as pms
import os, sys
from glob import glob
from PIL import Image 
import keras
import tensorflow

## Resize images

In [ ]:
# imgs = glob('./hair_images/*')

# def resize(imgs):
#     for img in imgs:
#         filename = img.split('/')[2]
#         filename = filename.split('.')[0]
#         if filename == 'image_urls':
#             continue
#         if os.path.exists('./resized_hair_images/'+filename+'_resized.jpg') == True:
#             continue
#         im = Image.open(img)
#         imResize = im.resize((300,400), Image.ANTIALIAS)
#         imResize.save('./resized_hair_images/'+filename+'_resized.jpg', 'JPEG', quality=90)

# resize(imgs)

## Find hair via contours

In [ ]:
# min_YCrCb = np.array([0,133,77],np.uint8)
# max_YCrCb = np.array([255,173,127],np.uint8)

In [ ]:
# img = cv2.imread('wavy.jpg')

# imageYCrCb = cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
# skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)

# contours, hierarchy = cv2.findContours(skinRegion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Draw the contour on the source image
# for i, c in enumerate(contours):
#     area = cv2.contourArea(c)
#     if area > 9000:
#         cv2.drawContours(img, contours, i, (0, 255, 0), 3)

# # Display the source image
# cv2.imwrite('output1.jpg', img)
# plt.figure(figsize=(12,12))
# plt.imshow(img[:,:,::-1])
# plt.show()

## Find faces

In [ ]:
# face_cascade = cv2.CascadeClassifier('/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/cv2/data/haarcascade_frontalface_default.xml')
# eye_cascade = cv2.CascadeClassifier('/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/cv2/data/haarcascade_eye.xml')

In [ ]:
# img = cv2.imread('hair.jpg')
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=3)

# for (x,y,w,h) in faces:
#     cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
#     roi_gray = gray[y:y+h, x:x+w]
#     roi_color = img[y:y+h, x:x+w]
#     eyes = eye_cascade.detectMultiScale(roi_gray,minSize=(100, 100), maxSize=(150,150))
#     for (ex,ey,ew,eh) in eyes:
#         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

# cv2.imwrite('output2.jpg', img)
# plt.figure(figsize=(12,12))
# plt.imshow(img[:,:,::-1])
# plt.show()

In [ ]:
# img = cv2.imread('hair.jpg')
# img = cv2.cvtColor(img ,cv2.COLOR_BGR2GRAY)
# cv2.imwrite('blackwhite.jpg', img)

# (segmented_image, labels_image, number_regions) = pms.segment(img, spatial_radius=6, 
#                                                               range_radius=4.5, min_density=1500)


In [ ]:
# plt.figure(figsize=(12,12))
# plt.imshow(segmented_image)
# plt.show()

# cv2.imwrite('blackwhite_seg.jpg', segmented_image)

## Playing with edge detection

In [ ]:
# import cv2
# import cv2
# import numpy as np
# from matplotlib import pyplot as plt

In [ ]:
# img = cv2.imread('hair_images/903ff49749.jpg',0)
# edges = cv2.Canny(img,100,250)

# plt.figure(figsize=(20,20))
# plt.subplot(121),plt.imshow(img,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(edges,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

# plt.show()


In [ ]:
# def auto_canny(image, sigma=0.33):
# 	# compute the median of the single channel pixel intensities
# 	v = np.median(image)
     
# 	# apply automatic Canny edge detection using the computed median
# 	lower = int(max(0, (1.0 - sigma) * v))
# 	upper = int(min(255, (1.0 + sigma) * v))
# 	edged = cv2.Canny(image, lower, upper)
 
# 	# return the edged image
# 	return edged

# imagePath = 'hair_images/903ff49749.jpg'
# image = cv2.imread(imagePath)
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blurred = cv2.GaussianBlur(gray, (3, 3), 0)


In [ ]:

# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blurred = cv2.GaussianBlur(gray, (3, 3), 0)
 
# # apply Canny edge detection using a wide threshold, tight
# # threshold, and automatically determined threshold
# wide = cv2.Canny(blurred, 10, 200)
# tight = cv2.Canny(blurred, 225, 250)
# auto = auto_canny(blurred)

In [ ]:
# plt.figure(figsize=(20,20))
# plt.subplot(121),plt.imshow(img,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(wide,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

# plt.show()

## Classifying masked Reddit images via CNN

In [1]:
#import sys
#sys.path.append('./flaskexample/')
import importlib
import model_image
importlib.reload(model_image)

Using TensorFlow backend.
/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/joanna/anaconda2

<module 'model_image' from '/Users/joanna/Desktop/Insight/CurlyHair/model_image.py'>

In [2]:
from glob import glob
import pandas as pd
import numpy as np
import os

In [3]:
images = glob('masked_images/twentyfour/*')


In [4]:
new_df = pd.DataFrame(columns=['file_path', 'class', 'pred'])
#new_df.head()

In [5]:
count = 0
for i,im in enumerate(images):
    print(i)
    fil = './hair_images/'+im.split('/')[2]
    print(fil)
    
    #row = img_df[img_df['file_path'].str.match(fil)]
    #if row.empty:
    #    count += 1
    #    continue

    label = model_image.predict_class(im)
    #print(label)
    
    #ind = row.index[0]
    #img_df['class'].loc[ind] = label[0]
    #img_df['pred'].loc[ind] = label[1]
    new_df = new_df.append({'file_path' :fil , 'class' :label[0], 'pred':label[1]} , ignore_index=True)
    
new_df.to_csv('image_urls_labels24.csv')

0
./hair_images/ad3f504ea1.jpg
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
1
./hair_images/aedd05030a.jpg
2
./hair_images/ad9dd80e0b.jpg
3
./hair_images/ba8eace3ed.jpg
4
./hair_images/b760763943.jpg
5
./hair_images/bade9cc0f8.jpg
6
./hair_images/adee46862b.jpg
7
./hair_images/ae9bf1b545.jpg
8
./hair_images/abe71ac7ce.jpg
9
./hair_images/b99133b264.jpg
10
./hair_images/b479986fa3.jpg
11
./hair_images/ac8ee58466.jpg
12
./hair_images/b731853146.jpg
13
./hair_images/aebdb060a6.jpg
14
./hair_images/ba40eddeb7.jpg
15
./hair_images/ae667cdca6.jpg
16
./hair_images/ba352ebdcf.jpg
17
./hair_images/ae12821717.jpg
18
./hair_images/ac6d04a5ae.jpg
19
./hair_images/ad79abe401.jpg
20
./hair_images/ba3a80e812.jpg
21
./hair_images/b82721765b.jpg
22
./hair_images/aeef5dafca.jpg
23
./hair_images/b219693927.jpg
24
./hair_images/aba085cdd4.jpg
25
./hair_images/af85ad